In [1]:
a = {"a":2,"b":3}
a

{'a': 2, 'b': 3}

In [6]:
for i,fn in a.items():
    print(i)
    print(fn)

a
2
b
3


In [12]:
import numpy as np
x = np.random.rand(3)
y = x.astype(np.float32)
np.result_type(y)

dtype('float32')